In [ ]:
using Plots 
using CSV
using Tables
using DataFrames
using GLM
using Measures
plotly()

In [ ]:
colors=palette(:seaborn_colorblind)

In [ ]:
res = DataFrame(CSV.File("clinicalAndMetrics.csv"))

In [ ]:
res.synMetric=res.peakSyn./res.peakEIA;
res.COONumber=res."COO_by_Any_GEP_+_Nanostring(string)"
res[res.COONumber.=="GCB",:COONumber].=string(1)
res[res.COONumber.=="ABC",:COONumber].=string(2)
res[res.COONumber.=="Unclassified",:COONumber].=string(3)
res[res.COONumber.=="N/A",:COONumber].=string(3)


res.OS=res."Overall_Survival_Months(number)";

In [ ]:
# This is the last deceased patient before the first living one
lowOS_res=res[res.OS .<= 33.9, :];

In [ ]:
pred_all = DataFrame(OS = 0:1:127);

In [ ]:
function plotSyn(OS_col, metric_col, ylabel, n)    
        p = plot(legend=:outertopright)
            plot!(pred.OS, predFrame.prediction, label="model", linewidth=2,color=colors[1])
            plot!(pred.OS, predFrame.prediction, label="model", linewidth=0, color=colors[1],fillalpha=0.3,
                    ribbon = (predFrame.prediction .- predFrame.lower, predFrame.upper .- predFrame.prediction))
            plot!(OS_col, metric_col, label="data", seriestype=:scatter, xlab="Overall survival (months)",
                   ylab=ylabel, color=colors[4],markersize=5,markerstrokewidth=0)
            plot!(legend=false)
            #plot!(yticks=(log10.(collect(range(100,1000,10))),collect(range(100,1000,10))))
            if n == 130
                plot!(xticks=((collect(range(0,130,14))),collect(range(0,130,14))))
            else
                plot!(xticks=((collect(range(0,30,7))),collect(range(0,30,7))))
            end
            plot!(size=(900,500))
            #plot!(left_margin = 10mm)
            plot!(tickfontsize=12, xguidefontsize=18, yguidefontsize=18)
        display(p)
    return(p)
end

In [ ]:
function plotSynCluster(OS_col, metric_col, ylabel, n,colorCol)    
        p = plot(legend=:outertopright)
            plot!(pred.OS, predFrame.prediction, label=false, linewidth=2,color=colors[1])
            plot!(pred.OS, predFrame.prediction, label=false, linewidth=0, color=colors[1],fillalpha=0.3,
                    ribbon = (predFrame.prediction .- predFrame.lower, predFrame.upper .- predFrame.prediction))
            plot!(OS_col, metric_col, label=colorCol, seriestype=:scatter, xlab="Overall survival (months)",
                   ylab=ylabel, color=colorCol,markersize=5,markerstrokewidth=0,palette =colors)
            plot!(legend=false)
            #plot!(yticks=(log10.(collect(range(100,1000,10))),collect(range(100,1000,10))))
            if n == 130
                plot!(xticks=((collect(range(0,130,14))),collect(range(0,130,14))))
            else
                plot!(xticks=((collect(range(0,30,7))),collect(range(0,30,7))))
            end
            plot!(size=(900,500))
            #plot!(left_margin = 10mm)
            plot!(tickfontsize=12, xguidefontsize=18, yguidefontsize=18)
        display(p)
    return(p)
end

## Low overall survival results - synergy metric

In [ ]:
### OS(M) vs synergy 
pred = DataFrame(OS = 0:1:34);
model = lm(@formula(synMetric ~ OS), lowOS_res)
predFrame = predict(model, pred, level = 0.95,interval = :confidence);
plotSyn(lowOS_res."Overall_Survival_Months(number)", lowOS_res.synMetric, "Synergy score", 34)
println("r squared = "*string(r2(model)))

In [ ]:
synDF=copy(lowOS_res)

cutoff=mean(synDF.synMetric)
synDF=copy(synDF)
synDF2 = copy(synDF)
synDF3 = copy(synDF)
synDF=filter(:synMetric => n -> n > 0.5, synDF)
synDF2=filter(:synMetric => n -> n > 0.25 && (n<=0.5), synDF2)
synDF3=filter(:synMetric => n -> n <= 0.25, synDF3)

@df synDF boxplot([1], :OS, label="syn metric >0.5",linewidth=2)
@df synDF dotplot!([1], :OS, marker=(:blue,stroke(0)), label="",mode = :density)

@df synDF2 boxplot!([2], :OS, label="syn metric[0.25,0.5)",linewidth=2)
@df synDF2 dotplot!([2], :OS, marker=(:green,stroke(0)), label="",xlim=(0,4),ylim=(0,40),ylabel="OS",mode = :density)

@df synDF3 boxplot!([3], :OS, label="syn metric<0.25",linewidth=2)
@df synDF3 dotplot!([3], :OS, marker=(:orange,stroke(0)), label="",xlim=(0,4),ylim=(0,50),ylabel="OS",mode = :density)

xticks!([1], [""])
plot!(legend=:outertopright)
#pval=pvalue(ApproximateTwoSampleKSTest(synDF.newNormSyn, synDF2.newNormSyn); tail = :both)
pval=pvalue(MannWhitneyUTest(synDF.OS, synDF2.OS); tail = :both)
annotate!(1.5,40,"p="*string(round(pval,digits=3)))
pval=pvalue(MannWhitneyUTest(synDF2.OS, synDF3.OS); tail = :both)
annotate!(2.5,40,"p="*string(round(pval,digits=3)))
pval=pvalue(MannWhitneyUTest(synDF.OS, synDF3.OS); tail = :both)
annotate!(2,50,"p="*string(round(pval,digits=3)))

xticks!([1,2], [""])

In [ ]:
display(colors[1:5])
println("Cluster: 1,2,3,4,5")

In [ ]:
pred = DataFrame(OS = 0:1:34);
model = lm(@formula(synMetric ~ OS), lowOS_res)
predFrame = predict(model, pred, level = 0.95,interval = :confidence);
plotSynCluster(lowOS_res."Overall_Survival_Months(number)", lowOS_res.synMetric, "Synergy score", 30,lowOS_res."Cluster(number)")
println("r squared = "*string(r2(model)))

In [ ]:
display(colors[1:3])
println("COO: GCB,ABC,UNC")

In [ ]:
pred = DataFrame(OS = 0:1:34);
model = lm(@formula(synMetric ~ OS), lowOS_res)
predFrame = predict(model, pred, level = 0.95,interval = :confidence);
plotSynCluster(lowOS_res."Overall_Survival_Months(number)", lowOS_res.synMetric, "Synergy score", 30,parse.(Int64, lowOS_res.COONumber))
println("r squared = "*string(r2(model)))

In [ ]:
display(colors[1:6])
println("IPI: 1  2  3  4  5")

In [ ]:
pred = DataFrame(OS = 0:1:34);
model = lm(@formula(synMetric ~ OS), lowOS_res)
predFrame = predict(model, pred, level = 0.95,interval = :confidence);
plotSynCluster(lowOS_res."Overall_Survival_Months(number)", lowOS_res.synMetric, "Synergy score", 30,parse.(Int64, lowOS_res."IPI(number)"))
println("r squared = "*string(r2(model)))

# using peak cell number

In [ ]:
pred = DataFrame(OS = 0:1:34);
model = lm(@formula(peakCN~ OS), lowOS_res)
predFrame = predict(model, pred, level = 0.95,interval = :confidence);
plotSyn(lowOS_res."Overall_Survival_Months(number)", lowOS_res.peakCN, "peak CN ", 30)
println("r squared = "*string(r2(model)))

In [ ]:
lowOS_res.log10peakCN=log10.(lowOS_res.peakCN)
pred = DataFrame(OS = 0:1:34);
model = lm(@formula(log10peakCN~ OS), lowOS_res)
predFrame = predict(model, pred, level = 0.95,interval = :confidence);
plotSyn(lowOS_res."Overall_Survival_Months(number)", lowOS_res.log10peakCN, "log10 peak CN", 30)
println("r squared = "*string(r2(model)))

In [ ]:
Pkg.add("HypothesisTests")

In [ ]:
using Statistics
using StatsPlots
using Query
using HypothesisTests
using Statistics
using DataFrames